In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

print(tf.__version__)

1.9.0


In [2]:
class_mapping = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 105s 1us/step


In [ ]:
x_predict = x_test[:44]
y_predict = y_test[:44]

num_class = 10

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_class)
y_test = tf.keras.utils.to_categorical(y_test, num_class)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
data_gen.fit(x_train)

In [ ]:
def build_cnn_model():
    input_layer = tf.keras.layers.Input(shape=(32, 32, 3), name="input_layer")
    use_bias = True

    # Conv1
    conv = tf.keras.layers.Conv2D(32,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(input_layer)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)

    # Conv2
    conv = tf.keras.layers.Conv2D(32,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)

    # MaxPooling1
    max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(activation)
    dropout = tf.keras.layers.Dropout(0.2)(max_pool)

    # Conv3
    conv = tf.keras.layers.Conv2D(64,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(dropout)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)

    # Conv4
    conv = tf.keras.layers.Conv2D(64,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)

    # MaxPooling2
    max_pool = tf.keras.layers.MaxPooling2D()(activation)
    dropout = tf.keras.layers.Dropout(0.3)(max_pool)

    # Conv5
    conv = tf.keras.layers.Conv2D(128,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(dropout)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)
    # Conv6
    conv = tf.keras.layers.Conv2D(128,
                                  kernel_size=(3, 3),
                                  padding='same',
                                  use_bias=use_bias,
                                  activation=None)(activation)
    bn = tf.keras.layers.BatchNormalization(epsilon=1e-06, axis=-1, momentum=0.9)(conv)
    activation = tf.keras.layers.Activation(tf.nn.relu)(bn)

    # MaxPooling3
    max_pool = tf.keras.layers.MaxPooling2D()(activation)
    dropout = tf.keras.layers.Dropout(0.4)(max_pool)

    # Dense Layer
    flatten = tf.keras.layers.Flatten()(dropout)

    # Softmax Layer
    output = tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')(flatten)

    return tf.keras.Model(inputs=input_layer, outputs=output)

In [ ]:
model = build_cnn_model()
model.summary()

In [ ]:
opt_rms = tf.keras.optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=opt_rms, metrics=['accuracy'])

In [ ]:
batch_size = 64
log_dir = "training"

tbCallback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=50,
                                            write_graph=True, write_grads=True, batch_size=batch_size,
                                            write_images=True)

checkpoint_path = os.path.join(log_dir, "cp.ckpt")

In [ ]:
# save weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    period=5)

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003

    return lrate

In [ ]:
model.fit_generator(data_gen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size, epochs=125,
                    verbose=1, validation_data=(x_test, y_test),
                    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule), cp_callback, tbCallback])

result = model.predict(x_predict / 255.0)

In [ ]:
pos = 1
for img, lbl, predict_lbl in zip(x_predict, y_predict, result):
    output = np.argmax(predict_lbl, axis=None)
    plt.subplot(4, 11, pos)
    plt.imshow(img)
    plt.axis('off')
    if output == lbl:
        plt.title(class_mapping[output])
    else:
        plt.title(class_mapping[output] + "/" + class_mapping[lbl[0]], color='#ff0000')
    pos += 1

In [ ]:
plt.show()

In [ ]:
scores = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\nTest accuracy: %.3f loss: %.3f' % (scores[1] * 100, scores[0]))